### Code to compute war and peace frame scores and victim and villain socres for news headlines

In [ ]:
### Make sure to specify the path to the trained model (model can be trained using 1_Benchmark_dataset)

In [ ]:
import pandas as pd
import numpy as np
import sys
import torch

from sklearn.model_selection import train_test_split
from transformers import BertModel, BertTokenizer
from transformers import AutoModel, AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, ModernBertModel

from transformers import get_scheduler

import torch.nn as nn
from torch.optim import AdamW

from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
import torch.nn.functional as F

pd.set_option('display.max_colwidth', None)

In [ ]:
data = pd.read_csv('data/GazaNews.csv')

In [ ]:
# data.columns

In [ ]:
class LanguageModel(nn.Module):
    
    def __init__(self, attr, siamese_training, num_classes = 2):
        
        super(LanguageModel, self).__init__()
        torch.manual_seed(0)
        self.model = AutoModel.from_pretrained(model_name, output_hidden_states = True, output_attentions = True)
        
        self.dropout = nn.Dropout(p = 0.15)
        self.classifier = nn.Linear(self.model.config.hidden_size, num_classes)
        
    def forward(self, input_ids, attention_masks):
        
        # x = self.model(input_ids = input_ids, attention_mask = attention_masks).last_hidden_state.mean(dim=1)
        emb = self.model(input_ids = input_ids, attention_mask = attention_masks)
        
        emb = emb.last_hidden_state[:, 0, :]
    
        x = self.dropout(emb)
        x = self.classifier(x)
        
        return x
    

In [ ]:
label = 'emotive'
### Specify the location of the trained model
model = torch.load(f'models/benchmark/roberta-base/Label_{label}.pth')

In [ ]:
model_name = 'roberta-base'
# model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device('cuda:0')
model.to(device)
'Model Loaded!'

In [ ]:
def get_dataset(sent):
    
    # sent = mask_entities(sent)
    
    tokens = tokenizer(sent, return_tensors="pt", padding=True, truncation=True)
    # tokens = tokenizer(sent, return_tensors="pt", padding='max_length', truncation=True)
    data = TensorDataset(tokens['input_ids'], tokens['attention_mask'])

    return DataLoader(data, batch_size = 32)

def get_label(dataloader):
    
    pred_labels = []
    logits = []
    logit = None
    for batch in dataloader:
        
        input_ids, attention_mask = [x.to(device) for x in batch]
        
        with torch.no_grad():
            logit = model(input_ids, attention_mask)
            
        preds = torch.argmax(logit, dim=-1)
        logits.extend(logit.detach().cpu().numpy())
        # print(preds)
        # sys.exit()
        pred_labels.extend(preds.detach().cpu().numpy())
    
    return logits, pred_labels

In [ ]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

def mask_entities(sentences: pd.Series) -> pd.Series:
    def mask(sentence):
        doc = nlp(sentence)
        masked = sentence
        for ent in doc.ents:
            masked = masked.replace(ent.text, "<mask>")
        return masked
    return sentences.apply(mask)

# def get_data(attr):
    
#     df = pd.read_csv(f'../../../../dataset/dataset_per_class/Label_{attr}.csv')
#     # print(df.columns)
    
#     df['headline'] = mask_entities(df['headline'])
    
#     return df

# df = get_data('1')

headlines = mask_entities(data.headline)

train_dataloader = get_dataset(headlines.tolist())

In [ ]:
# def get_data(attr):
    
#     df = pd.read_csv(f'../../../../dataset/dataset_per_class/Label_{attr}.csv')
#     df['headline'] = mask_entities(df['headline'])
    
#     return df

In [ ]:
pred_scores, _ = get_label(train_dataloader)

In [ ]:
softmax_scores = F.softmax(torch.tensor(pred_scores), dim=-1)

In [ ]:
softmax_scores_ = softmax_scores.numpy()

In [ ]:
df.columns

In [ ]:
def get_data(attr):
    
    df = pd.read_csv(f'data/Label_{attr}.csv')
    
    return df

# attr = '4c'
df = get_data(label)

class_mapping = dict({'emotive':'1', 'effects': '4a', 'source': '4b', 'context': '4c', 'villain': '3a', 'victim': '3b'})
lab = class_mapping[label]

mapping = dict(zip(df['label'], df[f'Label_{lab}']))
# data[f'Label_{attr}'] = data[f'Lab_{label}'].replace(mapping)
cols = ['score_' + str(l[1]) for l in sorted(mapping.items(), key = lambda x: x[0])]

In [ ]:
softmax_scores_df = pd.DataFrame(softmax_scores_, columns = cols)

In [ ]:
pd.concat((data[['id', 'indexed_date', 'language',
        'media_name', 'media_url', 'publish_date', 'headline', 'url', 'labels',
        'entity_labels', 'lab', 'entities', 'lang', 'pal_ent', 'hamas_ent',
        'isr_ent', 'other_ent', 'date', f'Lab_{label}', 
       f'Label_{label}']], softmax_scores_df), axis = 1).to_csv(f'results/news_analysis/Label_{label}.csv')

In [ ]:
data.shape, data.columns

In [ ]:
# pred_labels_ = torch.argmax(softmax_scores, dim=-1)

In [ ]:
# data[f'Lab_{label}'] = predictions

In [ ]:
# data[data['Lab_2a'] == 1]

In [ ]:
data.columns

In [ ]:
# data.to_csv('../ASP/github_code/data/GazaNews_war_and_peace.csv')